In [1]:
# !nvidia-smi

In [1]:
%load_ext autoreload
%autoreload 2

# %cd PROTES

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
# !pip install -r requirements.txt
# !pip install transformers

In [3]:
import math

import jax
jax.config.update('jax_enable_x64', True)
import numpy as np
import torch
from torch.nn.functional import softmax, log_softmax
from torch.optim import AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer


from protes import protes_gpt, protes

device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Creating something

In [4]:
softmax = torch.nn.Softmax(dim=-1)
logsoftmax = torch.nn.LogSoftmax(dim=-1)

In [5]:
def custom_nll_loss(logits, I):
    """
    logits: 1 x (1 + d) x n
    I:      1 x (1 + d)
    where 1 is added becaus of SOS token in the beggining
    how to deal with bs > 1 i don't understand so far
    """
    loss = 0
    bs, seq_len = I.shape
    
    P = logsoftmax(logits)
    for j in range(bs):
        for i in range(1, seq_len):
            loss += P[j, i, I[j, i]] 

    loss = torch.mean(loss, dim=0)
    return -loss



#     scores = model.compute_transition_scores(
#                 sequences=outputs.sequences,
#                 scores=outputs.scores,
#             )
#     loss = scores[idx][:, -1].mean()

In [6]:
def trainer(model, func, d, m, k, k_top, is_max, n):
    best_func_value = -torch.inf if is_max else torch.inf
    best_idx = None
    
#     prompt = torch.tensor([tokenizer.encode("<|endoftext|>")]).to(device)
    prompt = torch.tensor([[n+1]]).to(device)
    
    for i in range(math.ceil(m/k)):
        outputs =  model.generate(
            prompt,
            attention_mask=torch.ones_like(prompt).to(device),
            max_new_tokens=d,

            do_sample=True,
            num_beams=1,
            num_return_sequences=k,
            top_k=0,
            temperature=0.6,
            length_penalty=0,

            output_scores=True,
            return_dict_in_generate=True, 
            renormalize_logits=True, 
            output_hidden_states=True,
            
            pad_token_id=n+2,
            eos_token_id=n+2,
            bos_token_id=n+1,
            
            # MIGHT BE A PROBLEM
#             pad_token_id=tokenizer.eos_token_id
        )
        I = outputs.sequences

        val, idx = torch.topk(func(I[:, 1:]), largest=is_max, k=1)
        

        if is_max and (val > best_func_value):
            best_func_value = val
            best_idx = I[idx, 1:]
        
        if not is_max and (val < best_func_value):
            best_func_value = val
            best_idx = I[idx, 1:]
        
        _, idxes = torch.topk(func(I), largest=is_max, k=k_top)
        batch_of_best_I = I[idxes]
        
        optimizer.zero_grad()
        logits = model.forward(batch_of_best_I, attention_mask=torch.ones_like(batch_of_best_I).to(device)).logits
        
#         logits = model(batch_of_best_I, attention_mask=None).logits
        
        loss = criterion(logits, batch_of_best_I)
        loss.backward()

        optimizer.step()

        if i % 10 == 0:
            print('batch {} loss: {} best_value {}'.format(i, loss.item(), best_func_value.item()))
    

    return best_func_value, best_idx, model

### Rosenbrock

In [7]:
a = -3
b = 3

n = 1000 #50257 #len(tokenizer)
d = 2
k = 32
m = 50000

k_top = 8
is_max = False
log = True

def f_rosenbrock(I):
#     I = (I - 1) / (n) * (b - a) + a
    I = I / (n - 1) * (b - a) + a
    f = (1 - I[:, 0]) ** 2 + 100 * (I[:, 1] - I[:, 0] ** 2) ** 2 
    return f

def f_rosenbrock_jax(I):
    I = (I) / (n - 1) * (b-a) + a
    f = (1 - I[:, 0]) ** 2 + 100 * (I[:, 1] - I[:, 0] ** 2) ** 2 
    return f

func = f_rosenbrock

In [8]:
# model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=0, bos_token_id=0).to(device)
# word_embeddings = model.transformer.wte.weight
# word_embeddings.shape
# model.resize_token_embeddings(1 + n)
# word_embeddings1 = model.transformer.wte.weight
# word_embeddings1.shape
# word_embeddings[0] - word_embeddings1[0]

In [9]:
# tokenizer = AutoTokenizer.from_pretrained("gpt2", add_special_tokens=True)
# tokenizer.bos_token_id
# # add the EOS token as PAD token to avoid warnings
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.encode("<|endoftext|>")

# model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)  # pad_token_id=0, bos_token_id=0
model = AutoModelForCausalLM.from_pretrained("gpt2", bos_token_id=n+1, pad_token_id=n+2).to(device)
model.resize_token_embeddings(n + 2)
model.train()

word_embeddings1 = model.transformer.wte.weight
word_embeddings1.shape

torch.Size([1002, 768])

Process ForkProcess-27:
Process ForkProcess-18:
Process ForkProcess-8:
Process ForkProcess-4:
Process ForkProcess-6:
Process ForkProcess-31:
Process ForkProcess-19:
Process ForkProcess-20:
Process ForkProcess-5:
Process ForkProcess-2:
Process ForkProcess-25:
Process ForkProcess-22:
Process ForkProcess-29:
Process ForkProcess-1:
Process ForkProcess-23:
Process ForkProcess-10:
Process ForkProcess-11:
Process ForkProcess-24:
Process ForkProcess-32:
Process ForkProcess-21:
Process ForkProcess-30:
Process ForkProcess-17:
Process ForkProcess-3:
Process ForkProcess-28:
Process ForkProcess-7:
Process ForkProcess-9:
Process ForkProcess-16:
Process ForkProcess-26:
Process ForkProcess-15:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last)

In [10]:
idx = torch.tensor([
    [0, 1, 102],
    [0, 234, 88],
    [0, 11, 22]
])

q = model.forward(idx.to(device), labels=None, attention_mask=torch.ones_like(idx).to(device))

print(q.keys())
print(q.logits.shape)
logits = q.logits
logsoftmax(logits[0][0])

P = logsoftmax(logits)
P.shape

odict_keys(['logits', 'past_key_values'])
torch.Size([3, 3, 1002])


torch.Size([3, 3, 1002])

In [11]:
# q.logits.shape
# q.loss
# logsoftmax(logits[0][0]).sum()
custom_nll_loss(logits, idx)

tensor(50.4577, device='cuda:0', grad_fn=<NegBackward0>)

In [12]:
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = custom_nll_loss  # torch.nn.NLLLoss(reduction="mean")

In [13]:
best_func_value, best_idx, model = trainer(model=model, func=func, d=d, m=m, k=k, k_top=k_top, is_max=is_max, n=n)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


batch 0 loss: 134.2897186279297 best_value 58.94215393066406
batch 10 loss: 96.93930053710938 best_value 0.26437389850616455
batch 20 loss: 26.30117416381836 best_value 0.0965811088681221
batch 30 loss: 3.6898975372314453 best_value 0.0965811088681221
batch 40 loss: 0.7264370322227478 best_value 0.0965811088681221
batch 50 loss: 0.0348549447953701 best_value 0.03397068381309509
batch 60 loss: 0.09617158770561218 best_value 0.03397068381309509
batch 70 loss: 0.33873969316482544 best_value 0.03397068381309509
batch 80 loss: 0.14325936138629913 best_value 0.03397068381309509
batch 90 loss: 0.042591262608766556 best_value 0.03397068381309509
batch 100 loss: 0.5464655756950378 best_value 0.03397068381309509
batch 110 loss: 0.1867319494485855 best_value 0.03397068381309509
batch 120 loss: 0.09350378066301346 best_value 0.03397068381309509
batch 130 loss: 0.05046858638525009 best_value 0.03397068381309509
batch 140 loss: 0.15787756443023682 best_value 0.03397068381309509
batch 150 loss: 0.113

In [14]:
best_func_value

tensor([0.], device='cuda:0')

In [15]:
best_idx

tensor([[666, 666]], device='cuda:0')

In [17]:
best_X = (best_idx) / (n - 1) * (b-a) + a
best_X

tensor([[1., 1.]], device='cuda:0')

In [18]:
# Original PROTES

i_opt, y_opt, ll_list = protes(f=func,
                          d=d, n=n, k=k, m=m, log=log, is_max=is_max,
                        k_top=k_top, k_gd=1, lr=1e-3)

print(f"i_opt = {i_opt}, x_opt = {i_opt / (n-1) * (b-a) + a} f_opt = {y_opt}")

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


protes > m 3.2e+01 | t 3.845e+00 | y  2.0643e+01
protes > m 6.4e+01 | t 3.865e+00 | y  9.1681e-01
protes > m 3.5e+02 | t 3.922e+00 | y  8.6135e-01
protes > m 4.2e+02 | t 3.935e+00 | y  1.1805e-01
protes > m 4.8e+02 | t 3.947e+00 | y  3.9693e-02
protes > m 2.5e+03 | t 4.335e+00 | y  4.1826e-03
protes > m 1.6e+04 | t 6.842e+00 | y  3.6202e-05
protes > m 2.1e+04 | t 7.848e+00 | y  3.6202e-05
protes > m 4.8e+04 | t 1.295e+01 | y  0.0000e+00
protes > m 5.0e+04 | t 1.339e+01 | y  0.0000e+00 <<< DONE
i_opt = [666 666], x_opt = [1. 1.] f_opt = 0.0


In [19]:
# Original PROTES lr smaller

i_opt, y_opt, ll_list = protes(f=func,
                          d=d, n=n, k=k, m=m, log=log, is_max=is_max,
                        k_top=k_top, k_gd=1, lr=1e-5)

print(f"i_opt = {i_opt}, x_opt = {i_opt / (n-1) * (b-a) + a} f_opt = {y_opt}")

protes > m 3.2e+01 | t 2.027e+00 | y  2.0643e+01
protes > m 6.4e+01 | t 2.034e+00 | y  9.1681e-01
protes > m 4.2e+02 | t 2.102e+00 | y  1.1805e-01
protes > m 2.1e+03 | t 2.431e+00 | y  7.4703e-02
protes > m 2.5e+03 | t 2.506e+00 | y  2.9979e-02
protes > m 8.4e+03 | t 3.625e+00 | y  3.6867e-03
protes > m 2.9e+04 | t 7.611e+00 | y  6.1046e-04
protes > m 4.2e+04 | t 1.003e+01 | y  1.4637e-04
protes > m 5.0e+04 | t 1.170e+01 | y  1.4637e-04 <<< DONE
i_opt = [664 662], x_opt = [0.98798799 0.97597598] f_opt = 0.00014637034775424478


### Simple 3D

In [28]:
a = -6
b = 6

m = 50_000
# n0 = 3439

n = 1001
d = 3
k = 256

k_top = 64

is_max = False
log = True

def f_3d_squares(I):
    I = I / (n - 1) * (b - a) + a
    x = I[:, 0]
    y = I[:, 1]
    z = I[:, 2]
    f = (x - 5) ** 2 + (y - 2) ** 2 + (z + 1) ** 2
    return f


# def f_3d_squares_jax(I):
#     I = (I) / (n-1) * (b-a) + a
#     x = I[:, 0]
#     y = I[:, 1]
#     z = I[:, 2]
#     f = (x - 5) ** 2 + (y - 2) ** 2 + (z + 1) ** 2
#     return f

func = f_3d_squares

In [30]:
model = AutoModelForCausalLM.from_pretrained("gpt2", bos_token_id=n+1, pad_token_id=n+2).to(device)
model.resize_token_embeddings(n + 2)
model.train()
optimizer = AdamW(model.parameters(), lr=1e-6)
criterion = custom_nll_loss

best_func_value, best_idx, model = trainer(model=model, func=func, d=d, m=m, k=k, k_top=k_top, is_max=is_max, n=n)

print(f"PROTES_GPT: i_opt = {best_idx}, x_opt = {(best_idx) / (n-1) * (b-a) + a} f_opt = {best_func_value}")

batch 0 loss: 1488.1719970703125 best_value 20.11132049560547
batch 10 loss: 1412.528076171875 best_value 7.103664398193359
batch 20 loss: 1327.9609375 best_value 4.863025188446045
batch 30 loss: 1228.7393798828125 best_value 3.4460651874542236
batch 40 loss: 1216.9857177734375 best_value 3.4460651874542236
batch 50 loss: 1059.252197265625 best_value 0.2780632972717285
batch 60 loss: 883.8412475585938 best_value 0.2780632972717285
batch 70 loss: 622.2764282226562 best_value 0.2780632972717285
batch 80 loss: 373.7566223144531 best_value 0.2780632972717285
batch 90 loss: 234.7258758544922 best_value 0.2780632972717285
batch 100 loss: 174.2445068359375 best_value 0.2780632972717285
batch 110 loss: 142.3838348388672 best_value 0.2780632972717285
batch 120 loss: 83.68028259277344 best_value 0.2780632972717285
batch 130 loss: 84.15532684326172 best_value 0.2780632972717285
batch 140 loss: 73.4128189086914 best_value 0.2780632972717285
batch 150 loss: 58.91435241699219 best_value 0.2780632972

In [31]:
# Original PROTES

i_opt, y_opt, ll_list = protes(f=func,
                          d=d, n=n, k=k, m=m, log=log, is_max=is_max,
                        k_top=k_top, k_gd=1, lr=1e-5)

print(f"i_opt = {i_opt}, x_opt = {i_opt / (n-1) * (b-a) + a} f_opt = {y_opt}")

protes > m 2.6e+02 | t 3.609e+00 | y  7.4813e-01
protes > m 1.0e+03 | t 3.676e+00 | y  2.1806e-01
protes > m 3.3e+03 | t 3.889e+00 | y  2.5296e-02
protes > m 5.0e+04 | t 8.337e+00 | y  2.5296e-02 <<< DONE
i_opt = [918 663 404], x_opt = [ 5.016  1.956 -1.152] f_opt = 0.025295999999999735


In [32]:
i_opt, y_opt, ll_list = protes(f=func,
                          d=d, n=n, k=k, m=m, log=log, is_max=is_max,
                        k_top=k_top, k_gd=1, lr=1e-1)

print(f"i_opt = {i_opt}, x_opt = {i_opt / (n-1) * (b-a) + a} f_opt = {y_opt}")

protes > m 2.6e+02 | t 3.571e+00 | y  7.4813e-01
protes > m 1.0e+03 | t 3.644e+00 | y  1.2398e-01
protes > m 2.0e+03 | t 3.737e+00 | y  3.1776e-02
protes > m 1.0e+04 | t 4.458e+00 | y  2.4096e-02
protes > m 1.5e+04 | t 4.901e+00 | y  7.5840e-03
protes > m 2.7e+04 | t 6.043e+00 | y  3.9360e-03
protes > m 2.8e+04 | t 6.161e+00 | y  1.8720e-03
protes > m 3.6e+04 | t 6.983e+00 | y  1.1040e-03
protes > m 4.2e+04 | t 7.625e+00 | y  8.6400e-04
protes > m 4.6e+04 | t 7.968e+00 | y  3.3600e-04
protes > m 5.0e+04 | t 8.376e+00 | y  3.3600e-04 <<< DONE
i_opt = [918 667 416], x_opt = [ 5.016  2.004 -1.008] f_opt = 0.0003360000000000113


### nd

In [33]:
a = 0
b = 6


n = 100
d = 5
m = 100_000

k = 256

k_top = 64

is_max = False
log = True

def f_nd_squares(I):
    I = (I-1) / (n) * (b-a) + a
    bias = torch.arange(I[0].shape[0]) + 1
    I = I - bias.to(I.device)
    f = I[:, None, : ] @ I[:, :, None]
    f = f.squeeze(-1).squeeze(-1)
    return f


func = f_nd_squares


In [34]:
model = AutoModelForCausalLM.from_pretrained("gpt2", bos_token_id=n+1, pad_token_id=n+2).to(device)
model.resize_token_embeddings(n + 2)
model.train()
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = custom_nll_loss

In [35]:
idx = torch.tensor([
    [0, 1, 102],
    [0, 234, 88],
    [0, 11, 22]
])

In [36]:
func(idx).shape

torch.Size([3])

In [37]:
best_func_value, best_idx, model = trainer(model=model, func=func, d=d, m=m, k=k, k_top=k_top, is_max=is_max, n=n)

print(f"PROTES_GPT: i_opt = {best_idx}, x_opt = {(best_idx) / (n-1) * (b-a) + a} f_opt = {best_func_value}")

batch 0 loss: 1553.803955078125 best_value 3.8356001377105713
batch 10 loss: 44.10208511352539 best_value 3.8356001377105713
batch 20 loss: 9.548042297363281 best_value 3.8356001377105713
batch 30 loss: 0.4761412739753723 best_value 3.8356001377105713
batch 40 loss: 0.04874479025602341 best_value 3.8356001377105713
batch 50 loss: 0.19058306515216827 best_value 3.8356001377105713
batch 60 loss: 0.2525249719619751 best_value 3.8356001377105713
batch 70 loss: 0.11834334582090378 best_value 3.8356001377105713
batch 80 loss: 0.10485654324293137 best_value 3.8356001377105713
batch 90 loss: 0.0910305380821228 best_value 3.8356001377105713
batch 100 loss: 0.40059894323349 best_value 3.8356001377105713
batch 110 loss: 0.40419235825538635 best_value 3.8356001377105713
batch 120 loss: 0.3527459502220154 best_value 3.8356001377105713
batch 130 loss: 0.07445167005062103 best_value 3.8356001377105713
batch 140 loss: 1.5046732425689697 best_value 3.8356001377105713
batch 150 loss: 0.08892916142940521

In [38]:
# Original PROTES

def f_nd_squares_jax(I):
    I = I / (n-1) * (b-a) + a
    bias = np.arange(I[0].shape[0]) + 1
    I = I - bias
    f = I[:, None, : ] @ I[:, :, None]
    f = f.squeeze(-1).squeeze(-1)
    return f

func_jax = f_nd_squares_jax

i_opt, y_opt, ll_list = protes(f=func_jax,
                          d=d, n=n, k=k, m=m, log=log, is_max=is_max,
                        k_top=k_top, k_gd=1, lr=1e-3)

print(f"i_opt = {i_opt}, x_opt = {i_opt / (n-1) * (b-a) + a} f_opt = {y_opt}")

protes > m 2.6e+02 | t 4.449e+00 | y  9.5409e-01
protes > m 2.6e+03 | t 4.521e+00 | y  5.7576e-01
protes > m 6.7e+03 | t 4.648e+00 | y  5.5372e-01
protes > m 1.4e+04 | t 4.864e+00 | y  5.3535e-01
protes > m 3.3e+04 | t 5.458e+00 | y  4.4353e-01
protes > m 3.6e+04 | t 5.544e+00 | y  4.0312e-01
protes > m 4.4e+04 | t 5.798e+00 | y  3.9578e-01
protes > m 4.8e+04 | t 5.909e+00 | y  2.1947e-01
protes > m 7.5e+04 | t 6.737e+00 | y  1.4968e-01
protes > m 1.0e+05 | t 7.538e+00 | y  1.4968e-01 <<< DONE
i_opt = [17 30 49 65 88], x_opt = [1.03030303 1.81818182 2.96969697 3.93939394 5.33333333] f_opt = 0.1496786042240585


### some drafts

In [20]:
idx = torch.tensor([
    [0, 1, 102],
#     [0, 234, 88]
])

idx = torch.tensor([[0]])
q = model.forward(idx.to(device), labels=None, attention_mask=torch.ones_like(idx).to(device))

print(q.keys())
print(q.logits.shape)
logits = q.logits
logsoftmax(logits[0][0])

P = logsoftmax(logits)
P.shape

odict_keys(['logits', 'past_key_values'])
torch.Size([1, 1, 1003])


torch.Size([1, 1, 1003])

In [6]:
# !pip show transformers

In [ ]:
n = 10

# tokenizer = AutoTokenizer.from_pretrained("gpt2", add_special_tokens=True)
# # add the EOS token as PAD token to avoid warnings
# tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=0).to(device)
model.resize_token_embeddings(1 + n)
model.eval()
2-2

In [8]:
# # encode context the generation is conditioned on
# model_inputs = tokenizer('1 + 2', return_tensors='pt').to(device)
# print(model_inputs)

# # generate 40 new tokens
# greedy_output = model.generate(**model_inputs, max_new_tokens=40)
# print(f"Greedy output {greedy_output}")

# print("Output:\n" + 100 * '-')
# print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

In [9]:
q = torch.tensor([[0]]).to(device)
# inp = {"input_ids": q, "attention_mask"}
# multinomial sampling
greedy_output = model.generate(q, max_new_tokens=2, num_beams=1,
                               output_scores=True,
                               return_dict_in_generate=True, 
                               renormalize_logits=True, 
                            num_return_sequences=5, do_sample=True, 
                                output_hidden_states=True,
                                top_k=0,
                               temperature=0.6,
                               length_penalty=0,
                              )
#scores = torch.cat(greedy_output.scores)
idx = greedy_output.sequences

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [10]:
idx

tensor([[0, 0, 1],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], device='cuda:0')

In [11]:
softmax = torch.nn.Softmax(dim=-1)

In [12]:
z = model.forward(q)
print(z.keys())

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


odict_keys(['logits', 'past_key_values'])


In [13]:
z.logits.shape

torch.Size([1, 1, 11])

In [14]:
scores = model.compute_transition_scores(
    sequences=greedy_output.sequences,
    scores=greedy_output.scores,

#     beam_indices=greedy_output.beam_indices,
)

In [15]:
softmax(scores)

tensor([[0.8508, 0.1492],
        [0.7090, 0.2910],
        [0.7090, 0.2910],
        [0.7090, 0.2910],
        [0.7090, 0.2910]], device='cuda:0')

In [16]:
greedy_output.keys()

odict_keys(['sequences', 'scores', 'hidden_states'])

In [17]:
greedy_output.sequences[-1]

tensor([0, 0, 0], device='cuda:0')

In [18]:
greedy_output.scores

(tensor([[  0.0000, -21.3255, -32.1786, -33.3579, -22.9971, -26.4076, -17.0534,
          -24.9093, -21.2638, -22.4059, -21.0662],
         [  0.0000, -21.3255, -32.1786, -33.3579, -22.9971, -26.4076, -17.0534,
          -24.9093, -21.2638, -22.4059, -21.0662],
         [  0.0000, -21.3255, -32.1786, -33.3579, -22.9971, -26.4076, -17.0534,
          -24.9093, -21.2638, -22.4059, -21.0662],
         [  0.0000, -21.3255, -32.1786, -33.3579, -22.9971, -26.4076, -17.0534,
          -24.9093, -21.2638, -22.4059, -21.0662],
         [  0.0000, -21.3255, -32.1786, -33.3579, -22.9971, -26.4076, -17.0534,
          -24.9093, -21.2638, -22.4059, -21.0662]], device='cuda:0'),
 tensor([[-0.8904, -1.7407, -5.4421, -5.5557, -4.5824, -5.7360, -2.2392, -3.1334,
          -1.5789, -3.7138, -4.4431],
         [-0.8904, -1.7407, -5.4421, -5.5557, -4.5824, -5.7360, -2.2392, -3.1334,
          -1.5789, -3.7138, -4.4431],
         [-0.8904, -1.7407, -5.4421, -5.5557, -4.5824, -5.7360, -2.2392, -3.1334,
    

In [20]:
greedy_output.scores[1].shape

torch.Size([5, 11])

In [21]:
greedy_output.hidden_states[0][0].shape

torch.Size([5, 1, 768])

In [22]:
torch.exp(greedy_output.scores[0][0]).sum()

tensor(1., device='cuda:0')

In [23]:
greedy_output.scores[0].shape

torch.Size([5, 11])

In [24]:
greedy_output.hidden_states[0][0].shape

torch.Size([5, 1, 768])

In [25]:
probs = greedy_output.sequences_scores()

AttributeError: 'SampleDecoderOnlyOutput' object has no attribute 'sequences_scores'

In [26]:
P = greedy_output.scores

In [27]:
P[0].shape

torch.Size([5, 11])

In [28]:
probs

NameError: name 'probs' is not defined

In [40]:
model.eval()
2

2

In [41]:
q = torch.tensor([[0]]).to(device)

outputs =  model.generate(
            q, 
            max_new_tokens=d,
#             trace_log_probs=True,
            do_sample=True,
            num_beams=1,
            num_return_sequences=k,
            top_k=0,
            temperature=0.6,
            length_penalty=0,

            output_scores=True,
            return_dict_in_generate=True, 
            renormalize_logits=True, 
            output_hidden_states=True,
        )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [42]:
with torch.no_grad():
    model.eval()
    model = model.to(device)
    q = q.to(device)
    outputs =  model.greedy_search(
                q, 
                max_new_tokens=d,
    #             trace_log_probs=True,
                do_sample=True,
                num_beams=1,
                num_return_sequences=k,
                top_k=0,
                temperature=0.6,
                length_penalty=0,

                output_scores=True,
                return_dict_in_generate=True, 
                renormalize_logits=True, 
                output_hidden_states=True,
            )

ValueError: If `eos_token_id` is defined, make sure that `pad_token_id` is defined.

In [40]:
outputs.keys()

odict_keys(['sequences', 'scores', 'hidden_states'])

In [41]:
p = outputs.scores[0]
idx = outputs.sequences
print(p.shape, idx.shape)

torch.Size([5, 50257]) torch.Size([5, 3])


In [42]:
scores = model.compute_transition_scores(
    sequences=outputs.sequences,
    scores=outputs.scores,

#     beam_indices=greedy_output.beam_indices,
)
scores[[1, 2]][:, -1]

tensor([0., 0.], device='cuda:0')

In [43]:
q

tensor([[0]], device='cuda:0')

In [19]:
# outputs.sequences
# I = outputs.sequences[:, 1:]
# func(I)

# _, idx = torch.topk(func(I), largest=is_max, k=k_top)

# idx

# func(I).argmin(2)

In [63]:
2

2

# Sources
- https://huggingface.co/blog/how-to-generate
- https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration
- https://github.com/huggingface/transformers/issues/3720
- https://discuss.huggingface.co/t/how-to-output-loss-from-model-generate/16999/7
- https://github.com/huggingface/transformers/issues/15552 **try to read**
- https://github.com/Vision-CAIR/MiniGPT-4/issues/129
- https://stackoverflow.com/questions/45196631/how-to-upload-a-cloned-git-repository-to-an-own-git-repository-on-github

## Drafts, thrash

In [ ]:


# generate_with_grad = undecorated(model.generate)
# model.generate_with_grad = MethodType(generate_with_grad, model)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
tokenizer = AutoTokenizer.from_pretrained('t5-small')
# 
input_ids = tokenizer("propose new indexes the previous were 30, 40", return_tensors="pt").input_ids
# input_ids = torch.tensor([[0]])
encoder_outputs = model.encoder(input_ids)

decoder_input_ids = torch.ones_like(input_ids)[:, :1] * model.config.decoder_start_token_id
model_kwargs = {"encoder_outputs": encoder_outputs}

outputs = model.greedy_search(decoder_input_ids,
                                encoder_outputs=encoder_outputs,
                                max_new_tokens=d,
                                do_sample=True,
                                num_beams=1,
                                num_return_sequences=k,
                                top_k=0,
                                temperature=0.6,
                                length_penalty=0,

                                output_scores=True,
                                return_dict_in_generate=True, 
                                renormalize_logits=True, 
                                output_hidden_states=True,
                               )

print("Output:", tokenizer.batch_decode(outputs.sequences))
# => prints `['<pad> Heute ist ein schöner Tag.</s>']



In [ ]:
from undecorated import undecorated
from types import MethodType

generate_with_grad = undecorated(model.generate)
model.generate_with_grad = MethodType(generate_with_grad, model)


sequences = model.greedy_search(decoder_input_ids, encoder_outputs=encoder_outputs, 
            max_new_tokens=d,

            do_sample=True,
            num_beams=1,
            num_return_sequences=k,
            top_k=0,
            temperature=0.6,
            length_penalty=0,

            output_scores=True,
            return_dict_in_generate=True, 
            renormalize_logits=True, 
            output_hidden_states=True,)

# print("Output:", tokenizer.batch_decode(sequences))
# => prints `['<pad> Heute ist ein schöner Tag.</s>']


encoder_outputs